labelimg 로 이미지 라벨링 하고,   
txt 로 저장한 좌표 정보를   
xml 로 좌표를 변경해서 생성해주는 프로그램

In [1]:
'''
함수부, 한번만 실행하세요
'''

from PIL import Image
import os
import progressbar

def yolo_txt2xml_coordinate(size, bbox):
    '''
    bbox = (txt_x, txt_w, txt_y, txt_h)
    xmin, xmax, ymin, ymax = yolo_txt2xml_coordinate((w, h), bbox)
    '''
    dw = 1./size[0]  # 이미지 가로 길이
    dh = 1./size[1]  # 이미지 세로 길이
    
    # x 좌표
    x_tmp = (bbox[0] / dw) * 2.0
    w_tmp = bbox[1] / dw      # x 좌표간 길이
    
    x1 = (x_tmp - w_tmp) / 2  # x 시작 좌표
    x2 = x1 + w_tmp           # x 끝 좌표
    
    # y 좌표
    y_tmp = (bbox[2] / dh) * 2.0
    h_tmp = bbox[3] / dh      # y 좌표간 길이
    
    y1 = (y_tmp - h_tmp) / 2  # y 시작 좌표
    y2 = y1 + h_tmp           # y 끝 좌표
    
    return x1, x2, y1, y2
    
def xml_coordinate_create(dest_path, img_name, size, bbox):
    '''
    xml 좌표 파일 생성
    '''
    # file_name.jpg 에서 file_name 만 분리
    file_name, ext = os.path.splitext(img_name)
    
    # /path/file_name.xml 생성
    path_name = os.path.join(dest_path, file_name) +'.xml'
    
    # <object> 태그 부분
    obj_tag = []
    for i in range(len(bbox[0])):
        tmp_str = [
            '\t<object>\n',
            '\t\t<name>'+ file_name +'</name>\n',
            '\t\t<pose>Unspecified</pose>\n',
            '\t\t<truncated>0</truncated>\n',
            '\t\t<difficult>0</difficult>\n',
            '\t\t<bndbox>\n',
            '\t\t\t<xmin>'+ str(bbox[0][i]) +'</xmin>\n',
            '\t\t\t<ymin>'+ str(bbox[1][i]) +'</ymin>\n',
            '\t\t\t<xmax>'+ str(bbox[2][i]) +'</xmax>\n',
            '\t\t\t<ymax>'+ str(bbox[3][i]) +'</ymax>\n',
            '\t\t</bndbox>\n',
            '\t</object>\n',
        ]
        obj_tag.extend(tmp_str)
    
    # 쓰기 파일 열기
    f = open(path_name, 'wt', encoding='utf-8')
    lines = [
        '<annotation>\n',
	'\t<folder>yolo_xml</folder>\n',
	'\t<filename>'+ img_name +'</filename>\n',
	'\t<path>'+ img_name +'</path>\n',
	'\t<source>\n',
		'\t\t<database>Unknown</database>\n',
	'\t</source>\n',
	'\t<size>\n',
		'\t\t<width>'+ str(size[0]) +'</width>\n',
		'\t\t<height>'+ str(size[1]) +'</height>\n',
		'\t\t<depth>3</depth>\n',
	'\t</size>\n',
	'\t<segmented>0</segmented>\n',
    ]
    
    tmp_list = ['</annotation>\n']
    obj_tag.extend(tmp_list)
    lines.extend(obj_tag )
    
    f.writelines(lines)
    f.close()
    
    return

def f_get_filename_img(dest_path='./', img_mode=True):
    '''
    설명: 특정 디렉토리에서 이미지 파일 여부에 따라 파일명 반환
    입력: 경로명, img_mode
    출력: 파일명 리스트
    옵션: img_mode = True -> 이미지 파일명 반환
         img_mode = False -> 이미지 파일이 아닌 파일의 파일명 반환
    예시: img_list = f_get_filename_img('./data', img_mode=True)
         print(img_list)
    '''
    import imghdr
    import os
    
    files_list = os.listdir(dest_path)
    
    ret_list = []  # 반환 할 리스트
    
    for file_name in files_list:
        path_name = os.path.join(dest_path, file_name)
        
        if imghdr.what(path_name) == None:  # 이미지 파일이 아니면
            if img_mode == False:
                ret_list.append(file_name)
        else:                               # 이미지 파일이면
            if img_mode == True:
                ret_list.append(file_name)
                
    return ret_list


In [3]:
'''
아래 예시대로 폴더 경로 설정 꼭 하세요. '/' 주의
file_path = './alyac_label/003~102/'

폴더 안에 알약 이미지 파일, 이미지 파일과 같은 이름의 txt 파일 있어야 합니다.

'''
file_path = './alyac_label/003~102/'
img_list = f_get_filename_img(file_path, img_mode=True)

for img_name in img_list:
    bar = progressbar.ProgressBar(maxval=len(img_list)).start()
    
    # txt 파일 존재 유무 확인
    txt_name, ext = os.path.splitext(img_name)
    txt_name = txt_name+'.txt'
    if os.path.isfile(os.path.join(file_path, txt_name)) == False:
        print('txt 파일이 없습니다: ', txt_name)
        continue
    
    # jpg 파일 읽기
    img_path_name = file_path + img_name
    img = Image.open(img_path_name)
    w = int(img.size[0])
    h = int(img.size[1])


    # txt 파일 읽기
    txt_path_name = file_path + txt_name
    with open(txt_path_name, 'r') as file:
        strings = file.readlines()

        x_min = []  # x 시작 좌표
        y_min = []  # y 시작 좌표
        x_max = []  # x 끝 좌표
        y_max = []  # y 끝 좌표

        for string in strings:
            tmp = string.replace('\n', '')
            tmp = tmp.split(' ')
            txt_x = float(tmp[1])
            txt_y = float(tmp[2])
            txt_w = float(tmp[3])
            txt_h = float(tmp[4])

            bbox = (txt_x, txt_w, txt_y, txt_h)
            xmin, xmax, ymin, ymax = yolo_txt2xml_coordinate((w, h), bbox)

            x_min.append(round(xmin))
            y_min.append(round(ymin))
            x_max.append(round(xmax))
            y_max.append(round(ymax))

            
    bbox = (x_min, y_min, x_max, y_max)
    xml_coordinate_create(file_path, img_name, (w, h), bbox)

bar.finish()

100% |########################################################################|
